In [2]:
#conda activate pyvizenv

In [3]:
#pip install yfinance

In [81]:
# Import Libraries
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import panel as pn
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
pn.extension('plotly')
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta, date
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()


In [24]:
# Laod .env environment
load_dotenv()


True

In [25]:
# Pull in API Key
api_key = os.getenv("glassnode_api")
type(api_key)


NoneType

In [26]:
# Define crypto currencies to pull
crypto_list = ["BTC"]


In [27]:
# Define Price URLs
price_url = 'https://api.glassnode.com/v1/metrics/market/price_usd'



## BTC API Data Pull

In [28]:
# Price API Request
btc_price_res = requests.get(price_url,
                      params={'a': 'BTC',
                              'i': '24h',
                              'api_key': api_key})


# Convert price to Pandas Dataframe, set index to time and clean up file
btc_price_df = pd.read_json(btc_price_res.text, convert_dates=['t'])
btc_price_df.columns = ['Date', 'BTC Price']
btc_price_df.set_index('Date', inplace=True) 




ValueError: Expected object or value

## BTC Data Table Debugging Cells

In [ ]:
btc_price_df

## BTC Daily Changes

In [ ]:
# Daily change in price column
btc_daily_price_pct_change = btc_price_df.pct_change(1)
btc_daily_price_pct_change.columns = ['BTC Daily Price Change']
btc_daily_price_pct_change = btc_daily_price_pct_change.dropna()
# btc_daily_price_pct_change
btc_daily_price_pct_change.tail()

## BTC Data Aggregating & Cleaning

In [ ]:
# Define all the different data frames into a list
btc_frames = [btc_price_df, btc_daily_price_pct_change]

# Concatenate all the dataframes into one
btc_data = pd.concat(btc_frames, axis=1, join="outer", ignore_index=False)
btc_data.tail()


# Load in DataNews CSV

In [69]:
# Generate DataFrame from CSV
bitcoin_df = pd.read_csv(Path('DataNews_Bitcoin.csv'))
bitcoin_df['Published Date'] = pd.to_datetime(bitcoin_df['Published Date'], exact = False, infer_datetime_format=True, format = '%Y/%m%d')
for i in bitcoin_df.index:
     bitcoin_df['Published Date'][i] = bitcoin_df['Published Date'][i].date()
bitcoin_df.set_index('Published Date', inplace = True)
bitcoin_df.index.names = ['Date']

In [79]:
bitcoin_df.head()

,Title,Content
Date,,
2016-05-05,Financial Blockchain Group PTDL Sees New Members,The blockchain-centric Post-Trade Distributed ...
2016-05-09,Bitcoin to Be Studied by the Australian Treasury,In an effort to understand and explore the pos...
2016-05-10,Peace & Stability Through The Blockchain: Info...,"This is part 5 of CCN.com’s series, “Peace & S..."
2016-05-16,"Central Banks Should Observe Blockchain, Says ...","The Deputy Governor of the Bank of Japan, the ..."
2016-05-17,Bitcoin Startup Blockchain Announces the Thund...,Bitcoin-centric startup Blockchain has announc...


In [82]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gss8640\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [84]:
# Create function to store the sentiment scores in DataFrame
btc_sentiments = []


for row in bitcoin_df:
    try:
        text = bitcoin_df["Content"]
        date = article["Date"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
    
        
        btc_sentiments.append({
            "text": text,
            "Date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
        })
        
    except AttributeError:
        pass

    

TypeError: string indices must be integers

In [17]:
btc_avg_sent = pd.DataFrame(headline_sentiment_summarizer_avg(btc_headlines))

In [18]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
btc_avg_sent.index = pd.to_datetime(dates)

In [19]:
# Merge with BTC returns
btc_sentiments = btc_daily_price_pct_change.join(btc_avg_sent).dropna(how="any")

# Display data
display(btc_sentiments)

,BTC Daily Price Change,btc_avg
Date,,
2021-04-03,-0.029451,-0.053460
2021-04-04,0.019254,0.157620
2021-04-05,0.002484,0.083360
2021-04-06,-0.011126,0.015850
2021-04-07,-0.034752,-0.043950
2021-04-08,0.037384,0.034785
2021-04-09,-0.001305,0.161575
2021-04-10,0.026581,0.101945
2021-04-11,0.007245,0.194425


In [20]:
# Correlate the headlines' sentiment to returns
btc_sentiments.corr()

,BTC Daily Price Change,btc_avg
BTC Daily Price Change,1.00000,0.10331
btc_avg,0.10331,1.00000


In [21]:
# Define features data
X = btc_sentiments.copy()
X = X.drop(columns=["btc_avg"])
X.head()

,BTC Daily Price Change
Date,
2021-04-03,-0.029451
2021-04-04,0.019254
2021-04-05,0.002484
2021-04-06,-0.011126
2021-04-07,-0.034752


In [22]:
# Define target data
y = btc_sentiments["btc_avg"].values
y = y.reshape(-1, 1)
y[:5]

array([[-0.05346],
       [ 0.15762],
       [ 0.08336],
       [ 0.01585],
       [-0.04395]])

In [25]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
# Create the scaler instance
X_scaler = StandardScaler()

In [27]:
# Fit the scaler
X_scaler.fit(X_train)

StandardScaler()

In [28]:
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)